In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets, models
from sklearn.model_selection import KFold
import numpy as np
import random
import os

In [10]:
import warnings

warnings.filterwarnings(action='ignore')

## Fix Random Seeds

In [11]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [15]:
set_seed(18)

## Parameter Setting

In [12]:
data_dir = 'fake_licenseplate'
batch_size = 32
num_epochs = 10
k_folds = 5
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((144, 48)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

## Model Design

In [13]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.features = models.resnet18(pretrained=True)
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        return x

## K-fold Cross Validation

In [14]:
kfold = KFold(n_splits=k_folds, shuffle=True)

results = {}

for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
    print(f'*** Fold {fold + 1} ***')

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_subsampler)

    model = SimpleCNN(num_classes=2)
    model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                images, labels = batch
                images, labels = images.to(device), labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = correct / total
        print(f'Fold {fold + 1}, Epoch {epoch + 1}, Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')

    results[fold] = val_acc

print(f'K-Fold Cross Validation results: {results}')
print(f'Average Accuracy: {np.mean(list(results.values())):.4f}')

*** Fold 1 ***
Fold 1, Epoch 1, Loss: 0.2607, Accuracy: 0.8909
Fold 1, Epoch 2, Loss: 0.2370, Accuracy: 0.9091
Fold 1, Epoch 3, Loss: 1.1363, Accuracy: 0.7636
Fold 1, Epoch 4, Loss: 0.0788, Accuracy: 0.9636
Fold 1, Epoch 5, Loss: 0.0693, Accuracy: 0.9636
Fold 1, Epoch 6, Loss: 0.0367, Accuracy: 0.9818
Fold 1, Epoch 7, Loss: 0.0264, Accuracy: 1.0000
Fold 1, Epoch 8, Loss: 0.0056, Accuracy: 1.0000
Fold 1, Epoch 9, Loss: 0.0014, Accuracy: 1.0000
Fold 1, Epoch 10, Loss: 0.0008, Accuracy: 1.0000
*** Fold 2 ***
Fold 2, Epoch 1, Loss: 54.1820, Accuracy: 0.3636
Fold 2, Epoch 2, Loss: 7.1800, Accuracy: 0.5818
Fold 2, Epoch 3, Loss: 2.4780, Accuracy: 0.6909
Fold 2, Epoch 4, Loss: 0.4123, Accuracy: 0.8545
Fold 2, Epoch 5, Loss: 0.3556, Accuracy: 0.9091
Fold 2, Epoch 6, Loss: 0.0474, Accuracy: 0.9636
Fold 2, Epoch 7, Loss: 0.0062, Accuracy: 1.0000
Fold 2, Epoch 8, Loss: 0.0378, Accuracy: 0.9818
Fold 2, Epoch 9, Loss: 0.0420, Accuracy: 0.9818
Fold 2, Epoch 10, Loss: 0.0196, Accuracy: 0.9818
*** Fol